# Challenge
Using selected questions from the 2012 and 2014 editions of the European Social Survey, address the following questions. Keep track of your code and results in a Jupyter notebook or other source that you can share with your mentor. For each question, explain why you chose the approach you did.

Here is the data file. And here is the codebook, with information about the variable coding and content.

In this dataset, the same participants answered questions in 2012 and again 2014.

Did people become less trusting from 2012 to 2014? Compute results for each country in the sample.<br>
Did people become happier from 2012 to 2014? Compute results for each country in the sample.<br>
Who reported watching more TV in 2012, men or women?<br>
Who was more likely to believe people were fair in 2012, people living with a partner or people living alone?<br>
Pick three or four of the countries in the sample and compare how often people met socially in 2014. Are there differences, and if so, which countries stand out?<br>
Pick three or four of the countries in the sample and compare how often people took part in social activities, relative to others their age, in 2014. Are there differences, and if so, which countries stand out?

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

## Data

In [2]:
ess = pd.read_csv('/Users/marenbeckman/Desktop/Data_Science/ESSdata_Thinkful.csv')


In [3]:
ess.head()

,cntry,idno,year,tvtot,ppltrst,pplfair,pplhlp,happy,sclmeet,sclact,gndr,agea,partner
0,CH,5.0,6,3.0,3.0,10.0,5.0,8.0,5.0,4.0,2.0,60.0,1.0
1,CH,25.0,6,6.0,5.0,7.0,5.0,9.0,3.0,2.0,2.0,59.0,1.0
2,CH,26.0,6,1.0,8.0,8.0,8.0,7.0,6.0,3.0,1.0,24.0,2.0
3,CH,28.0,6,4.0,6.0,6.0,7.0,10.0,6.0,2.0,2.0,64.0,1.0
4,CH,29.0,6,5.0,6.0,7.0,5.0,8.0,7.0,2.0,2.0,55.0,1.0


In [5]:
ess.shape

(8594, 13)

In [6]:
ess.describe()

,idno,year,tvtot,ppltrst,pplfair,pplhlp,happy,sclmeet,sclact,gndr,agea,partner
count,8.594000e+03,8594.000000,8586.000000,8580.000000,8555.000000,8569.000000,8563.000000,8579.000000,8500.000000,8584.000000,8355.000000,8577.000000
mean,3.954938e+04,6.500000,3.861985,5.559907,6.005143,5.319874,7.694616,5.192563,2.748941,1.497204,47.470736,1.384867
std,6.267259e+05,0.500029,2.019689,2.233700,2.129866,2.173449,1.735904,1.457643,0.905477,0.500021,18.397369,0.486592
min,1.000000e+00,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,15.000000,1.000000
25%,1.052000e+03,6.000000,2.000000,4.000000,5.000000,4.000000,7.000000,4.000000,2.000000,1.000000,33.000000,1.000000
50%,1.714000e+03,6.500000,4.000000,6.000000,6.000000,5.000000,8.000000,6.000000,3.000000,1.000000,47.000000,1.000000
75%,2.745000e+03,7.000000,5.000000,7.000000,8.000000,7.000000,9.000000,6.000000,3.000000,2.000000,62.000000,2.000000
max,1.100143e+07,7.000000,7.000000,10.000000,10.000000,10.000000,10.000000,7.000000,5.000000,2.000000,114.000000,2.000000


In [7]:
ess.groupby('cntry').describe()

agea                                                         gndr  \
        count       mean        std   min   25%   50%   75%    max   count   
cntry                                                                        
CH     1503.0  46.743180  18.632669  16.0  32.0  46.0  61.0   94.0  1546.0   
CZ     1280.0  46.301563  16.888995  15.0  34.0  46.0  59.0   91.0  1306.0   
DE       28.0  55.892857  16.067771  26.0  49.0  55.5  65.5   89.0    28.0   
ES     2360.0  47.874576  18.200719  16.0  34.0  47.0  61.0  103.0  2426.0   
NO     1436.0  46.536212  18.203938  15.0  32.0  46.0  60.0   94.0  1462.0   
SE     1748.0  49.040046  19.536782  15.0  32.0  50.0  65.0  114.0  1816.0   

                ...  tvtot         year                                     \
           mean ...    75%  max   count mean       std  min  25%  50%  75%   
cntry           ...                                                          
CH     1.513583 ...    4.0  7.0  1546.0  6.5  0.500162  6.0  6.0  6.5  7.0   
CZ     1.525268 ...    7.0  7.0  1316.0  6.5  0.500190  6.0  6.0  6.5  7.0   
DE     1.571429 ...    6.0  7.0    28.0  6.5  0.509175  6.0  6.0  6.5  7.0   
ES     1.504534 ...    6.0  7.0  2426.0  6.5  0.500103  6.0  6.0  6.5  7.0   
NO     1.466484 ...    5.0  7.0  1462.0  6.5  0.500171  6.0  6.0  6.5  7.0   
SE     1.476872 ...    5.0  7.0  1816.0  6.5  0.500138  6.0  6.0  6.5  7.0   

            
       max  
cntry       
CH     7.0  
CZ     7.0  
DE     7.0  
ES     7.0  
NO     7.0  
SE     7.0  

[6 rows x 96 columns]

## Did people become less trusting from 2012 to 2014? Compute results for each country in the sample.